# positive_graph
0. (sentiment_graphs copy)
1. import gml file
2. write to directed and undirected gml files
3. component and connectivity analysis
4. plot graph components and subgraphs

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from glob import glob

pd.set_option('display.mpl_style', 'default') 
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

#gml_files = glob('../output/network/*/*.gml')

def calculate_graph_inf(graph):
    graph.name = filename
    info = nx.info(graph)
    print info

def plot_graph(graph):
    info = nx.info(graph)
    print info
    plt.figure(figsize=(10,10))
    nx.draw_spring(graph, with_labels = True)

# start here

In [2]:
graph = nx.read_gml("../output_join/article_pos1.gml")
ugraph = graph.to_undirected()
U = graph.to_undirected(reciprocal=True)
e = U.edges()
ugraph.add_edges_from(e)

In [3]:
def drawIt(graph, what = 'graph'):
    nsize = graph.number_of_nodes()
    print "Drawing %s of size %s:" % (what, nsize)
    
    if nsize > 20:
        plt.figure(figsize=(10, 10))
        if nsize > 40:
            nx.draw_spring(graph, with_labels = True, node_size = 70, font_size = 12)
        else:
            nx.draw_spring(graph, with_labels = True)
    else:
        nx.draw_spring(graph, with_labels = True)
    plt.show()

def describeGraph(graph):
    components = sorted(nx.connected_components(graph), key = len, reverse = True)
    cc = [len(c) for c in components]
    subgraphs = list(nx.connected_component_subgraphs(graph))
    params = (graph.number_of_edges(),graph.number_of_nodes(),len(cc))
    print "Graph has %s nodes, %s edges, %s connected components\n" % params
    drawIt(graph)
    for sub in components:
        drawIt(graph.subgraph(sub), what = 'component')

## Components & connectivity

In [5]:
# list of connected components (sets of nodes), starting with largest
print "List of connected components =", [len(c) for c in sorted(nx.connected_components(ugraph), key=len, reverse=True)]

# generate connected components as subgraphs; Gc is largest component
subgraphs = list(nx.connected_component_subgraphs(ugraph))

List of connected components = [676, 15, 7, 6, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


### Greatest component

In [6]:
Gc = max(nx.connected_component_subgraphs(ugraph), key=len)
print "Size of greatest component =", len(Gc)

Size of greatest component = 676


Moody and White provide an algorithm for identifying k-components in a graph, which is based on Kanevsky’s algorithm for finding all minimum-size node cut-sets of a graph (implemented in all_node_cuts() function):

1. Compute node connectivity, k, of the input graph G.
2. Identify all k-cutsets at the current level of connectivity using Kanevsky’s algorithm.
3. Generate new graph components based on the removal of these cutsets. Nodes in a cutset belong to both sides of the induced cut.
4. If the graph is neither complete nor trivial, return to 1; else end.

In [7]:
# returns all minimum k cutsets of an undirected graph
# i.e., the set(s) of nodes of cardinality equal to the node connectivity of G
# thus if removed, would break G into two or more connected components
cutsets = list(nx.all_node_cuts(Gc))
print "# of cutsets =", len(cutsets)

# of cutsets = 148


In [8]:
# returns a set of nodes or edges of minimum cardinality that disconnects G
print "Min node cut =", nx.minimum_node_cut(Gc, s='vaccines', t='autism')
print "Min edge cut =", nx.minimum_edge_cut(Gc)

Min node cut = set([u'families', u'MMR vaccine', u'vaccinated children', u'autism risk', u'high-risk children', u'anti-vaccination', u'parents', u'scientists', u'children at higher risk for autism', u'children', u'vaccine', u'Jain study'])
Min edge cut = set([(u'anti-vaccination', u'time')])


In [9]:
nx.minimum_node_cut(Gc, s='vaccines', t='autism')

{u'Jain study',
 u'MMR vaccine',
 u'anti-vaccination',
 u'autism risk',
 u'children',
 u'children at higher risk for autism',
 u'families',
 u'high-risk children',
 u'parents',
 u'scientists',
 u'vaccinated children',
 u'vaccine'}

In [10]:
a = nx.minimum_edge_cut(Gc, s='autism', t='vaccines')
a

{(u'autism', u'MMR vaccine'),
 (u'autism', u'anti-vaccination'),
 (u'autism', u'children'),
 (u'autism', u'children at higher risk for autism'),
 (u'autism', u'families'),
 (u'autism', u'genetic predisposition'),
 (u'autism', u'harmful association'),
 (u'autism', u'high-risk children'),
 (u'autism', u'parents'),
 (u'autism', u'protective effect of vaccines'),
 (u'autism', u'scientists'),
 (u'autism', u'vaccinated children'),
 (u'autism', u'vaccines'),
 (u'children at increased genetic risk', u'autism risk'),
 (u'children with affected older sibling', u'autism risk'),
 (u'vaccinated children and unvaccinated children', u'autism risk'),
 (u'vaccinated high-risk children', u'Jain study'),
 (u'vaccine', u'immune response')}

In [11]:
labels = nx.get_edge_attributes(Gc,'edge')
edgelabels = {}
for e in labels.keys():
    e1 = e[0:2]
    edgelabels[e1]=labels[e]
edgelabels

{(u'2014-2015 FLULAVAL QUADRIVALENT flu vaccine',
  u'CDC'): u'does not recommend revaccination for people who received the recalled',
 (u'2014-2015 FLULAVAL QUADRIVALENT flu vaccine',
  u'GlaxoSmithKline'): u'recalling remaining doses of',
 (u'2014-2015 FLULAVAL QUADRIVALENT flu vaccine',
  u'influenza viruses'): u'is quadrivalent flu vaccine that protects against',
 (u'2014-2015 FLULAVAL QUADRIVALENT flu vaccine',
  u'reduced effectiveness'): u'potentially may have',
 (u'2014-2015 FLULAVAL QUADRIVALENT flu vaccine',
  u'safety concern'): u'reduced potency does not pose',
 (u'2014-2015 FLULAVAL QUADRIVALENT flu vaccine',
  u'vaccine potency'): u'fell below pre-specified limit prior to expiration of',
 (u'Afghanistan', u'polio vaccine opposition'): u'much in',
 (u'Age of Autism', u'Jain study'): u'already attacked',
 (u'Age of Autism', u'anti-vaccination'): u'is such',
 (u'American Medical Association',
  u'nonmedical reasons for exemptions'): u'members voted to eliminate',
 (u'America

In [12]:
for e in a:
    if edgelabels.has_key(e):
        print e,edgelabels[e]
    else:
        rev_e = e[::-1]
        print rev_e, edgelabels[rev_e]

(u'MMR vaccine', u'autism') researchers were unable to find any association with
(u'vaccinated children', u'autism') are somewhat less likely to be diagnosed with
(u'children', u'autism') one in 68 kids has some form of
(u'families', u'autism') remains challenge for
(u'anti-vaccination', u'autism') is driven by fears that shots cause
(u'parents', u'autism') who already have a child with autism seem even more concerned
(u'children at increased genetic risk', u'autism risk') no evidence it raises
(u'high-risk children', u'autism') were not more likely to develop
(u'harmful association', u'autism') none between MMR vaccine and
(u'immune response', u'vaccine') could be less potent after waiting longer to get
(u'scientists', u'autism') remains challenge for
(u'protective effect of vaccines', u'autism') may protect children from
(u'autism risk', u'children with affected older sibling') no evidence that MMR vaccine raised
(u'genetic predisposition', u'autism') makes more vulnerable to
(u'vacc

In [ ]:
# this takes forever
# average connectivity k of a graph G is the average of local node connectivity over all pairs of nodes of G

#nx.average_node_connectivity(Gc)

In [ ]:
# NEW SUMMARY

print "List of connected components =", [len(c) for c in sorted(nx.connected_components(ugraph), key=len, reverse=True)]
print "Size of greatest component =", len(Gc)
print "# of cutsets =", len(cutsets)
print "Min node cut =", nx.minimum_node_cut(Gc)
print "Min edge cut =", nx.minimum_edge_cut(Gc)